# Работа с контурами на изображении

## Немного про контуры

Детектирование контуров позволяет нам найти границу объекта на изображении и легко локализовать его. 

Обнаружение контуров используется в следующих задачах: 

* Детекция движения 
* Обнаружение объектов 
* Задачи сегментации 

## Что такое контуры 

Контур - это объединение всех точек по границе объекта. 

Обычно определённый контур относится к пограничным пикселям, которые имеют одинаковый цвет и интенсивность. 

К счастью, в OpenCV уже есть удобные функции: 

* [`cv2.findConrours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0)
* [`cv2.drawContours`](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc)

И также, два режима для обнаружения контуров: 

* [`cv2.CHAIN_APPROX_SIMPLE`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gga4303f45752694956374734a03c54d5ffa5f2883048e654999209f88ba04c302f5) - даёт только конечные точки, то есть своего рода аппроксимация 
* [`cv2.CHAIN_APPROX_NONE`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gga4303f45752694956374734a03c54d5ffaf7d9a3582d021d5dadcb0e37201a62f8) - даёт все точки 



## Как искать контур? 

1. Прочитать изображение и перевести его в оттенки серого 

    > Конвертация в оттенки серого нужна, чтобы в следующем шаге использовать пороговое значение, т.к. после конвертации мы получаем одноканальное изображение, то нам нужен порог только по одному каналу - это просто и удобно. А использование порогов по всем каналам в этой задаче только лишняя головная боль и работать нормально не будет, т.к. при поиске границ нам прежде всего важна интенсивность пикселей, чёрно-белая картинка намного лучше описывает эту информацию, нежели RGB. 

2. Применение бинарной маски (binary thresholding)

    > На этом шаге картинка превратится в бинарное изображение, где объект будет иметь белый цвет (значение = 1), а всё остальное - чёрный (значение = 0)

3. Использование функции [`cv2.findConrours`](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0) для поиска контуров 

4. Визуализация контуров - [`cv2.drawContours`](https://docs.opencv.org/4.x/d6/d6e/group__imgproc__draw.html#ga746c0625f1781f1ffc9056259103edbc)

    > Это важный шаг, т.к. всегда нужно понимать, что было найдено и нужна ли дополнительная обработка. 

## Немного вступления об аппроксимации

Давайте представим, что вы - это автономный беспилотный робот. 
Вы движетесь при помощи данных, полученных с разных датчиков: лидар, сонар, камеры и т.д. Вам постоянно приходится обрабатывать огромную кучу данных, чтобы успевать за изменениями вокруг. При этом, чем больше у вас есть данных, тем более точно вы сможете оценить обстановку. Но в то же время, тем горячее становится ваш вычислитель, так как приходится жарить данные с высокой скоростью. И тут мы спотыкаемся об такую штуку, как "вычислительная сложность и хранение данных". 

А что делать, если вы только что проснулись и ваши вычислительные ресурсы ещё очень малы? Ответ прост - Упрощать! 

> Либо говоря пафосным научным языком - "аппроксимировать" 



`Давайте считать, что наш робот автономен - т.е. не тащит за собой толстый кабель к ближайшей розетке, а везёт маленькую батарейку. Чем меньше энергии потребляет мозг робота, тем дольше последний проработает. А поэтому имеет смысл снизить потери на нагрев процессора, для чего снизить сложность алгоритмов или количество обрабатываемой информации`

Давайте представим, что у нас есть такой маршрут 


<p align="center">
    <img src="https://929687.smushcdn.com/2407837/wp-content/uploads/2021/10/normal_route.png?lossy=1&strip=1&webp=1" alt="drawing" width="600"/>
</p>

Допустим, нам дана ширина дороги и всякие другие её параметры. Опуская некоторые допущения, скажем, что повороты можно сгладить (т.е. аппроксимировать, т.е. упростить, чтобы уменьшить количество манёвров). 

Тогда наш "упрощённый" маршрут будет выглядить как-то так 

<p align="center">
    <img src="https://929687.smushcdn.com/2407837/wp-content/uploads/2021/10/rdp_route.gif?size=650x310&lossy=1&strip=1&webp=1" alt="drawing" width="600"/>
</p>

Если приглядеться повнимательнее, то заметно, как сглаживаются кривые линии. 

Это один из примеров аппроксимации контуров. 

## Что такое "аппроксимация контуров"? 

### Аппроксимация контуров на базе алгоритма Ramer-Douglas-Peucker (RDP)

Цель этого алгоритма - упросить ломанную (кривую) линию при помощи уменьшения количества её вершин с заданным пороговым значением. То есть мы как бы выкидываем некоторые близкие точки и получаем более гладкую линию, при этом не теряя важную информацию. 

> То есть если дорога вела в лес, она продолжить вести в лес, но с меньшим количеством поворотов. 

На гифке ниже Википедия рассказывает, как на самом деле работает алгоритм. 

<p align="center">
    <img src="https://lh4.googleusercontent.com/NJjqzOoXcJ9D_wa-6di-JDYms4CwhDVRaI3MURqcfjUQXiqII_TMQYHhqYGMJyvv-4JfCdrzmrnOjnqO_Swe2xDMx1PI3lMYi6oUw0L4mmtxoN28w-Rgpb1fI0rjMxKA1iYZaDFC=s0" alt="drawing" width="600"/>
</p>

У нас есть начальная и конечная точки кривой, алгоритм сначала найдёт вершину на максимальном расстоянии от линии, соединяющей две выбранные вершины. Если эта "дальняя" вершина находится на расстоянии меньшем, чем заданный порог, то мы автоматически пропускаем все вершины, которые лежат между и заменяем кривой участок прямой линией. 

Если "дальняя" вершина лежит дальше указанного порога, то мы рекурсивно повторяем алгоритм, двигаясь по исходной кривой. 

Давайте попробуем пощупать аппроксимацию.

In [ ]:
import os

import ipywidgets as widgets
from ipywidgets import interact

import numpy as np
import matplotlib.pyplot as plt 
import imutils
import cv2

Возьмём какое-нибудь простенькое контрастное изображение, чтобы выделить контур объекта. 

In [ ]:
fpath = os.path.join(os.pardir, "assets", "contour.png")
original_image = cv2.imread(fpath)
original_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2BGR)

plt.figure(figsize=[8, 8])
plt.imshow(original_image)
plt.title("Original Image")
plt.show()

Переведём картинку в оттенки серого и подберём пороговое значение так, чтобы объект был белым, а задний фон - чёрным. 

In [ ]:
gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)

@interact
def choose_threshold(
    low_thresh=widgets.IntSlider(value=50, min=0, max=255),
    high_thresh=widgets.IntSlider(value=255, min=0, max=255)
):
    thresholded_img = cv2.threshold(
        gray_image, low_thresh, high_thresh, cv2.THRESH_BINARY
    )[1]

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=[15, 10])
    
    ax[0].imshow(original_image)
    ax[0].set_title("Original Image")

    ax[1].imshow(thresholded_img, cmap="gray")
    ax[1].set_title("Thresholded Image")

    plt.show()

In [ ]:
# Как только подобрали пороги, которые вас устраивают, не забудьте обновить значения здесь
        # чтобы дальше использовать полученную маску
thresholded_img = cv2.threshold(gray_image, 15, 255, cv2.THRESH_BINARY)[1]

Итак, бинарная картинка у нас есть, давайте найдём на ней контур c помощью функции [cv2.findContours](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0). Мы будем использовать флаги: 

* [cv2.RETR_EXTERNAL](https://docs.opencv.org/4.x/d9/d8b/tutorial_py_contours_hierarchy.html), чтобы получить только внешние контуры, без иерархии (т.е. без вложенных контуров). 
* [cv2.CHAIN_APPROX_SIMPLE](https://docs.opencv.org/4.x/d3/dc0/group__imgproc__shape.html#gga4303f45752694956374734a03c54d5ffa5f2883048e654999209f88ba04c302f5), чтобы убрать лишине вершины, то есть это уже аппроксимация. 

In [ ]:
contours = cv2.findContours(
    thresholded_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
)
contours = imutils.grab_contours(contours)
max_cntr = max(contours, key=cv2.contourArea)

# Давайте нарисуем самый большой контур 
output_img = original_image.copy() 
cv2.drawContours(output_img, [max_cntr], -1, (0, 255, 0), 4)

print(f"Количество точек в контуре: {len(max_cntr)}")

plt.figure(figsize=[8, 8])
plt.imshow(output_img)
plt.title("Image with Contour")
plt.show()

Видим, что мы подобрали хороший порог и наш контур соответствует нашему объекту. Замечательно! 
Теперь давайте закатаем рукава и натравим на получившийся контур нашу собственную аппроксимацию. 


Давайте испробуем разные пороговые значения для алгоритма RDP. 

In [ ]:
eps_values = np.linspace(0.001, 0.05, 10)

@interact
def choose_threshold(
    eps_value=widgets.SelectionSlider(description="value", options=eps_values)
):
    peri = cv2.arcLength(max_cntr, True)
    approx = cv2.approxPolyDP(max_cntr, eps_value * peri, True)

    draw_img = original_image.copy()
    cv2.drawContours(draw_img, [approx], -1, (0, 255, 0), 3)
    print(f"Количество точек в контуре: {len(approx)}")

    plt.figure(figsize=[8, 8])
    plt.imshow(draw_img)
    plt.title("Image with Contour Approximation")
    plt.show()

Так можно и до абстракционизма лютого дойти. 
Так что всегда проверяйте промежуточные результаты, особенно в процессе работы с изображениями! 

## Полезные ссылки

* [PyImageSearch - Contour Approximation](https://www.pyimagesearch.com/2021/10/06/opencv-contour-approximation/)
* [Contour Detection](https://learnopencv.com/contour-detection-using-opencv-python-c/)